<a href="https://colab.research.google.com/github/harshit05092/GPT/blob/main/main_notebook_showing_successful_training_of_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
cfg = {
    "vocab_size": 50257, # No of vocab in GPT's BPE
    "context_length": 256, # The previous context which our gpt will consider
    "emb_dim": 768, # Number of dimensions that each token will be embedded in
    "n_heads": 12,  # No of attention heads
    "n_layers": 12, # No of layers
    "drop_rate": 0.1, # rate of dropping nn randomly
    "qkv_bias": False # Bias vector for query, key, value is False
}

In [ ]:
class Multiheadattention(nn.Module):

  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False) -> None:
    super().__init__() # Allow the inherited class paramenters to be initialized like backward pass and _parameters
    assert(d_out % num_heads == 0), \
    "d_out must be divisible by num_heads"
    self.d_out = d_out # Output dimensions that our transformer will output the code in
    self.num_heads = num_heads # Number of attention heads in our multihead attention layer
    self.head_dim = d_out // num_heads # Number of dimension each vector will be when then are output by each attention module
    self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias) # Weight matrix for query vector. din and dout are same generally
    self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias) # Weight matrix for key vectors
    self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias) # Weight matrix for value vectors
    self.out_proj = nn.Linear(d_out, d_out)
    self.dropout = nn.Dropout(dropout) # Dropout rate to prevent overfitting. For larger models it is set to 0.1
    self.register_buffer(
        "mask",
        torch.triu(torch.ones(context_length, context_length), diagonal = 1)
    )

  def forward(self, x):
    b, num_tokens, d_in = x.shape
    query = self.W_query(x)  # Matrix multiplication making query vector
    key = self.W_key(x) # Matrix multiplication for making key vector
    value = self.W_value(x) # Matrix multiplication for making value vector
    key = key.view(b, num_tokens, self.num_heads, self.head_dim)
    query = query.view(b, num_tokens, self.num_heads, self.head_dim)
    value = value.view(b, num_tokens, self.num_heads, self.head_dim)
    key = key.transpose(1, 2)
    value = value.transpose(1, 2)
    query = query.transpose(1, 2)
    attn_score = query @ key.transpose(2, 3)
    mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
    attn_score.masked_fill_(mask_bool, -torch.inf)
    attn_weights = torch.softmax(attn_score / key.shape[-1] ** 0.5, dim = -1)
    attn_weights = self.dropout(attn_weights)
    context_vec = (attn_weights @ value).transpose(1, 2)
    context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
    context_vec = self.out_proj(context_vec)
    return context_vec

In [ ]:
class GPTModule(nn.Module):

  def __init__(self, cfg) -> None:
    super().__init__()
    self.token_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"]) # Creates token embedding matrix of size vocab_size * emb_dim
    self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"]) # Creates token embedding matrix of size context_length * emb_dim
    self.drop_emb = nn.Dropout(cfg["drop_rate"]) # Sets Dropout to drop_rate

    self.trf_blocks = nn.Sequential(
        *[TransformerBlock(cfg)
        for _ in range(cfg["n_layers"])]
    )# Generates a list of transformer according to the number of layers.
     # If 12 then generate list to 12 transformer block stacked on each other

    self.final_norm = LayerNorm(cfg) # Set nomalization equal to layer normalization which is usually done in language models

    self.out_head = nn.Linear(
        cfg["emb_dim"], cfg["vocab_size"], bias=False
    ) # Final projection layer that turns each model embeddings into vocabulary logits for next token prediction

  def forward(self, in_idx):
    batch_size, seq_len = in_idx.shape
    token_embeds = self.token_emb(in_idx) # Gives token embeddingds in shape of in_idx * emb_dim
    pos_embeds = self.pos_emb(
        torch.arange(seq_len, device=in_idx.device)
        ) # Creates one-D tensor of size seq_len that represents absolute positioin in the sequence and ensures that it is placed into the
          # right CPU/GPU
    x = token_embeds + pos_embeds # Gives the postional token embeddings
    x = self.drop_emb(x) # Drops the 10%(0.1) of the input to prevent overfitting
    x = self.trf_blocks(x) # Passes them through transformer block
    x = self.final_norm(x) # Passes them through layer normalization
    logits = self.out_head(x) # Gives the next prediction
    return logits

In [ ]:
class TransformerBlock(nn.Module): # TransformerBlock class is a sub-block of nn.Module

  def __init__(self, cfg) -> None:
    super().__init__() # Initializes parameters from nn.Module
    self.attn = Multiheadattention(
        cfg["emb_dim"], cfg["emb_dim"], cfg["context_length"], cfg["drop_rate"], cfg["n_heads"], cfg["qkv_bias"]
    )# Sets the attention module to multiheadattention
    self.ff = FeedForward(cfg) # first projects the emb dim into 4 * emb_dim and then applies GeLU activation and then
                               # converts 4 * emb_dim into emb_dim
    self.norm1 = LayerNorm(cfg) # Applies layer normalization
    self.norm2 = LayerNorm(cfg) # Applies layer normalization
    self.dropout = nn.Dropout(cfg["drop_rate"]) # Sets dropout rate to 0.1

  def forward(self, x):
    shortcut = x # Shortcut connection for attention blocks
    x = self.norm1(x) # Applies layer normalization
    x = self.attn(x) # Applies multi-head attention
    x = self.dropout(x) # Applies dropout rate
    x = x + shortcut # Applies shortcut connection

    shortcut = x # Shorcut connection for FeedForward in which GeLU activation is applied
    x = self.norm2(x) # Applies layer normalization
    x = self.ff(x) # first projects the emb dim into 4 * emb_dim and then applies GeLU activation and then
                   # converts 4 * emb_dim into emb_dim
    x = self.dropout(x) # Applies dropout rate
    x = x + shortcut # Applies shortcut connection
    return x

In [ ]:
class LayerNorm(nn.Module):

  def __init__(self, cfg) -> None:
    super().__init__()
    self.scale = nn.Parameter(torch.ones(cfg["emb_dim"])) # Trainable parameter gamma
    self.shift = nn.Parameter(torch.zeros(cfg["emb_dim"])) # Trainable parameter beta

  def forward(self, x):
    mean = x.mean(dim=-1, keepdim=True) # Calculating mean of the layer
    var = x.var(dim=-1, keepdim=True, unbiased=False) # Calculating variance of the layer
    x = (x - mean) / torch.sqrt(var + 1e-5) # calculating standardized number. Dividing by 1e-5 to prevent diving by 0
    return x * self.scale + self.shift # Returning x * gamma + beta

In [ ]:
class GELU(nn.Module): # Calculates gaussian expression linear unit(GeLU)

  def __init__(self) -> None:
    super().__init__()

  def forward(self, x):
    return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))

In [ ]:
class FeedForward(nn.Module):

    def __init__(self, cfg) -> None:
      super().__init__()
      self.layers = nn.Sequential(
          nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
          GELU(),
          nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
      ) # Projects the embedding dimension into embedding_dimension * 4 and then applies GeLU activation and then gives
        # the output in original emb_dim

    def forward(self, x):
        return self.layers(x)

In [ ]:
torch.manual_seed(123)
model = GPTModule(cfg)
batch = torch.tensor([
    [6109, 3626, 6100, 345],
    [6109, 1110, 6622, 257]
])
out = model(batch)
print("Input batch: \n", batch)
print("Output batch: \n", out)

Input batch: 
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
Output batch: 
 tensor([[[-1.7955e-01,  2.8515e-01, -7.6131e-01,  ..., -4.8374e-01,
          -4.2503e-01, -1.7187e-01],
         [-6.2581e-01, -3.7480e-01, -9.7020e-01,  ...,  1.9168e-01,
          -1.3234e+00, -2.7643e-01],
         [ 5.1744e-01,  1.3866e-01,  2.4886e-01,  ...,  3.5054e-01,
          -7.7531e-02, -8.0042e-02],
         [-2.5664e-01, -6.9693e-01, -9.9479e-01,  ..., -4.4732e-02,
           6.1773e-02,  1.3467e-01]],

        [[-2.2379e-01,  1.1651e-01, -9.9836e-01,  ..., -1.5730e-01,
          -4.4800e-01, -2.8646e-02],
         [-8.7227e-01, -3.9389e-01, -1.1099e+00,  ...,  3.3035e-01,
          -9.2395e-02, -1.8597e-05],
         [ 4.5988e-01, -1.4274e-01, -1.2227e-01,  ...,  2.7490e-01,
           5.8297e-02, -8.9931e-02],
         [-6.2163e-01, -4.4854e-01, -4.7675e-01,  ..., -3.6519e-01,
           3.4402e-01, -3.8154e-01]]], grad_fn=<UnsafeViewBackward0>)


In [ ]:
def generate_text_sample(model, idx,
                         max_new_tokens, context_size):
  for _ in range(max_new_tokens):
    idx_cond = idx[:, -context_size:]
    with torch.no_grad():
      logits = model(idx_cond)
    logits = logits[:, -1, :]
    probas = torch.softmax(logits, dim=-1)
    idx_next = torch.argmax(probas, dim = -1, keepdim=True)
    idx = torch.cat((idx, idx_next), dim=1)
  return idx

In [ ]:
import tiktoken
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the determined device
model.to(device)

def text_to_token(tokenizer, text):
  encoded_text = tokenizer.encode(text)
  encoded_text = torch.tensor(encoded_text, dtype=torch.long).unsqueeze(0) # Added .unsqueeze(0) to make it 2D
  return encoded_text
def token_ids_to_text(tokenizer, token_ids):
  # If token_ids is 2D (batch_size, seq_len), convert to 1D before decoding
  if token_ids.dim() > 1:
    token_ids = token_ids.squeeze(0) # Assuming batch_size is 1 for decoding
  decoded_text = tokenizer.decode(token_ids.tolist())
  return decoded_text
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

# Ensure idx is on the correct device
initial_idx = text_to_token(tokenizer, start_context).to(device)

token_ids = generate_text_sample(
    model=model,
    idx = initial_idx,
    max_new_tokens=5,
    context_size=256
)
print("Output text:\n", token_ids_to_text(tokenizer, token_ids))

Output text:
 Every effort moves you?"
"Oh,


In [ ]:
input = torch.tensor(
    [[16833, 3626, 6100], # "every effort moves"
    [40,    1107, 588]] #"i really like"
).to(device)
target = torch.tensor(
    [[3626, 6100, 345], # "every effort moves you"
    [1107, 588, 11311]] # "really like chocolate"
).to(device)
with torch.no_grad():
  logits = model(input)
probas = torch.softmax(logits, dim=-1)
print(probas.shape)
token_ids = torch.argmax(probas, dim = -1, keepdim=True)
print(token_ids)
print(f"Target batch 1: {token_ids_to_text(tokenizer, target[0])}")
print(f"Output batch 1: {token_ids_to_text(tokenizer, token_ids[0].flatten())}")

torch.Size([2, 3, 50257])
tensor([[[ 832],
         [ 550],
         [4252]],

        [[ 367],
         [ 262],
         [ 757]]], device='cuda:0')
Target batch 1:  effort moves you
Output batch 1:  through had sun


In [ ]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], target[text_idx]]
text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], target[text_idx]]
print("Target 1 probabilites:", target_probas_1)
print("Target 2 probabilites:", target_probas_2)

Target 1 probabilites: tensor([1.0411e-04, 2.6049e-05, 1.0385e-05], device='cuda:0')
Target 2 probabilites: tensor([1.6884e-05, 5.1877e-05, 4.9942e-06], device='cuda:0')


In [ ]:
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print("Log probabilites:", log_probas)

Log probabilites: tensor([ -9.1700, -10.5555, -11.4751, -10.9891,  -9.8666, -12.2072],
       device='cuda:0')


In [ ]:
avg_log_probas = torch.mean(log_probas)
print("Average log probabilites:", avg_log_probas)
neg_avg_log_probas = -torch.mean(log_probas)
print("Negative average log probabilites:", neg_avg_log_probas)

Average log probabilites: tensor(-10.7106, device='cuda:0')
Negative average log probabilites: tensor(10.7106, device='cuda:0')


In [ ]:
logits_flat = logits.flatten(0, 1)
target_flat = target.flatten()
print("Logits shape:", logits_flat.shape)
print("Target shape:", target_flat.shape)

Logits shape: torch.Size([6, 50257])
Target shape: torch.Size([6])


In [ ]:
loss = torch.nn.functional.cross_entropy(logits_flat, target_flat)
perplexity = torch.exp(loss)
print("Loss:", loss)
print("Perplexity:", perplexity)

Loss: tensor(10.7106, device='cuda:0')
Perplexity: tensor(44828.9961, device='cuda:0')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/the-verdict.txt"
with open(file_path, "r") as file:
  text = file.read()
print(len(text))

20479


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import tiktoken
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

In [ ]:
train_ratio = 0.90
split_idx = int(len(text) * train_ratio)
train_text = text[:split_idx]
val_text = text[split_idx:]

In [ ]:
train_loader = create_dataloader_v1(
    train_text,
    batch_size = 2,
    max_length = 256,
    stride = cfg["context_length"],
    shuffle = True,
    drop_last = True,
    num_workers = 0
)
val_loader = create_dataloader_v1(
    val_text,
    batch_size = 2,
    max_length = 256,
    stride = cfg["context_length"],
    shuffle = False,
    drop_last = True,
    num_workers = 0
)
print("Train loader:", train_loader)
for x, y in train_loader:
    print(x.shape, y.shape)
print("Val loader:", val_loader)
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader: <torch.utils.data.dataloader.DataLoader object at 0x7e0027f70830>
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
Val loader: <torch.utils.data.dataloader.DataLoader object at 0x7e0027f50410>
torch.Size([2, 256]) torch.Size([2, 256])


In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    # Use PyTorch 2.9 or newer for stable mps results
    major, minor = map(int, torch.__version__.split(".")[:2])
    if (major, minor) >= (2, 9):
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
else:
    device = torch.device("cpu")


print(f"Using {device} device.")


model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Using cuda device.
Training loss: 10.987756623162163
Validation loss: 10.98848819732666


In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    # Fix: Pass tokenizer first, then start_context
    encoded = text_to_token(tokenizer, start_context).to(device)
    with torch.no_grad():
        token_ids = generate_text_sample(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(tokenizer, token_ids)
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

In [ ]:
import time
start_time = time.time()

torch.manual_seed(123)
model = GPTModule(cfg)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="I am", tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 9.818, Val loss 9.930
Ep 1 (Step 000005): Train loss 8.066, Val loss 8.336
I am,,,,,,,,,,,,,,.                                   
Ep 2 (Step 000010): Train loss 6.623, Val loss 7.053
Ep 2 (Step 000015): Train loss 6.047, Val loss 6.605
I am,,,,,, and,,,,,,, and,.                                 
Ep 3 (Step 000020): Train loss 5.532, Val loss 6.507
Ep 3 (Step 000025): Train loss 5.399, Val loss 6.389
I am, and, and, and the to the of the to the to the.                                  
Ep 4 (Step 000030): Train loss 4.895, Val loss 6.280
Ep 4 (Step 000035): Train loss 4.648, Val loss 6.304
I am a a--as of the picture the of the picture--as of the picture.             "I"I had the  "I the picture to the picture and I had been.
Ep 5 (Step 000040): Train loss 4.023, Val loss 6.165
I am he had been the "Oh, and he had been the fact of the last I felt to me--I had been.                        
Ep 6 (Step 000045): Train loss 3.625, Val loss 6.172
Ep 6 (Step 0000